In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import ISRPlotter as isrplot 

Welcome to JupyROOT 6.14/04


In [2]:
#isr_analysis["year"]["channel"]["level"]
#years=["2016","2017","2018"]
years=["2016"]
channels=["electron"]
levels=["DetUNFOLD", "DetUNFOLD_Acceptance", "FSRUNFOLD", "FSRUNFOLD_Acceptance"]

isr_analysis={}
for year in years :
    isr_analysis[year]=dict()
    for channel in channels :
        isr_analysis[year][channel]=dict()
        isr_analysis[year][channel]["detector"]= \
        isrplot.ISRPlotter("./inFiles/"+year+"/"+channel+"/unfold_input.root",
                           './config/'+year+'/'+channel+'/config_'+year+'_'+channel+'.json', False)
        
        isr_analysis[year][channel]["detector"].drawHistPlot("Mass", divde_by_bin_width = True, setLogy=True, showMeanValue=False)
        isr_analysis[year][channel]["detector"].drawHistPlot("Pt_0","Pt_1","Pt_2","Pt_3","Pt_4", 
                                                             divde_by_bin_width = True, setLogy=True, setLogx=False, showMeanValue=False)
        
        isr_analysis[year][channel]["detector"].drawISRPlot("DataBkgSub","Signal")
        
        doFit=False
        for level in levels :
            isr_analysis[year][channel][level]= \
            isrplot.ISRPlotter("./output/"+year+"/"+channel+"/"+level+"_"+channel+"_"+year+".root",
                               "./config/"+year+"/"+channel+"/config_"+level+"_"+channel+"_"+year+".json", False)
            
            isr_analysis[year][channel][level].drawHistPlot("Mass", divde_by_bin_width = True, setLogy=True, showMeanValue=False)
            isr_analysis[year][channel][level].drawHistPlot("Pt_0","Pt_1","Pt_2","Pt_3","Pt_4", 
                                                            divde_by_bin_width = True, setLogy=True, setLogx=False, showMeanValue=False)
            
            if level == "FSRUNFOLD_Acceptance" :
                doFit=True
            else : 
                doFit=False
                
            isr_analysis[year][channel][level].drawISRPlot("Data","Signal",do_linear_fit=doFit)
            


draw isr plot, do_linear_fit False
draw isr plot, do_linear_fit False
draw isr plot, do_linear_fit False
draw isr plot, do_linear_fit False
draw isr plot, do_linear_fit True
do log-linear fit!
Beta: [  3.80991055 -16.15500665]
Beta Std Error: [0.09423758 0.85114852]
Beta Covariance: [[ 0.02005195 -0.18098163]
 [-0.18098163  1.63575827]]
Residual Variance: 0.44288561632121765
Inverse Condition #: 0.00040935643056263084
Reason(s) for Halting:
  Sum of squares convergence


In [ ]:
raw_mc = isr_analysis["2016"]["electron"]["detector"].getRawDict("TotalMC")

In [ ]:
total_mc = raw_mc["Mass"]['total']['Nominal']['Nominal']['TH1'].Clone("total_mc")

In [ ]:
raw_data = isr_analysis["2016"]["electron"]["detector"].getRawDict("Data")

In [ ]:
data=raw_data["Mass"]["Data_Measured"]['Nominal']['Nominal']['TH1'].Clone("data")

In [ ]:
total_mc.Divide(data)

In [ ]:
total_mc.GetNbinsX()

In [ ]:
from root_numpy import hist2array, array2hist
temp_content, temp_binEdge=hist2array(total_mc, return_edges=True)

In [ ]:
temp_content

In [ ]:
refined_data = isr_analysis["2016"]["electron"]["FSRUNFOLD_Acceptance"].getRawDict("Data")

In [ ]:
refined_data["Pt_4"]["Data_Measured"]

In [ ]:
electron_eb_2016=isrplot.ISRPlotter("./inFiles/2016/electron/unfold_input_EB.root",'./config/2016/electron/config_2016_electron.json', False)
electron_eb_2017=isrplot.ISRPlotter("./inFiles/2017/electron/unfold_input_EB.root",'./config/2017/electron/config_2017_electron_.json', False)
electron_eb_2018=isrplot.ISRPlotter("./inFiles/2018/electron/unfold_input_EB.root",'./config/2018/electron/config_2018_electron_.json', False)

electron_ee_2016=isrplot.ISRPlotter("./inFiles/2016/electron/unfold_input_EE.root",'./config/2016/electron/config_2016_electron.json', False)
electron_ee_2017=isrplot.ISRPlotter("./inFiles/2017/electron/unfold_input_EE.root",'./config/2017/electron/config_2017_electron_.json', False)
electron_ee_2018=isrplot.ISRPlotter("./inFiles/2018/electron/unfold_input_EE.root",'./config/2018/electron/config_2018_electron_.json', False)

In [ ]:
electron_eb_2016.drawHistPlot("Pt_0","Pt_1","Pt_2","Pt_3","Pt_4", divde_by_bin_width = True, setLogy=True, setLogx=True, showMeanValue=False)
electron_eb_2017.drawHistPlot("Pt_0","Pt_1","Pt_2","Pt_3","Pt_4", divde_by_bin_width = True, setLogy=True, setLogx=True, showMeanValue=False)
electron_eb_2018.drawHistPlot("Pt_0","Pt_1","Pt_2","Pt_3","Pt_4", divde_by_bin_width = True, setLogy=True, setLogx=True, showMeanValue=False)

In [ ]:
electron_ee_2016.drawHistPlot("Pt_0","Pt_1","Pt_2","Pt_3","Pt_4", divde_by_bin_width = True, setLogy=True, setLogx=True, showMeanValue=False)
electron_ee_2017.drawHistPlot("Pt_0","Pt_1","Pt_2","Pt_3","Pt_4", divde_by_bin_width = True, setLogy=True, setLogx=True, showMeanValue=False)
electron_ee_2018.drawHistPlot("Pt_0","Pt_1","Pt_2","Pt_3","Pt_4", divde_by_bin_width = True, setLogy=True, setLogx=True, showMeanValue=False)

In [ ]:
electron_r9_2016=isrplot.ISRPlotter("./inFiles/2016/electron/unfold_input_r9.root",'./config/2016/electron/config_2016_electron.json', False)
electron_r9_2017=isrplot.ISRPlotter("./inFiles/2017/electron/unfold_input_r9.root",'./config/2017/electron/config_2017_electron_.json', False)
electron_r9_2018=isrplot.ISRPlotter("./inFiles/2018/electron/unfold_input_r9.root",'./config/2018/electron/config_2018_electron_.json', False)

In [ ]:
electron_r9_2016.drawHistPlotComparison(electron_r9_2017, electron_r9_2018, nominator_name="Data", denominator_name="Data", 
                                                                                variables=["Pt_0","Pt_1","Pt_2","Pt_3","Pt_4"], names=["2016", "2017", "2018"], markers=["<", ">"], colors=["blue", "red"],  
                                                                                divde_by_bin_width = True, setLogy=True, setLogx=True, showMeanValue=False, ratio_max=1.45, ratio_min=0.65)

In [ ]:
electron_r9_2016.drawISRPlots(electron_r9_2017, electron_r9_2018, 
                                                          names_in_objects=["Data","Data","Data" ], do_linear_fit=[False, False, False, False, False, False], 
                                                          labels=["2016 Br9E", "2017 Br9E", "2018 Br9E"], 
                                                          markers=["o-","<-",">-","o","<",">"], colors=["black","blue","red","black","blue","red"], 
                                                          facecolor_list=['black','blue','red','None', 'None', 'None'], ymin=16, ymax=33)

In [ ]:
#electron_eb.drawHistPlot("Pt_0","Pt_1","Pt_2","Pt_3","Pt_4", divde_by_bin_width = True, setLogy=True, setLogx=True, showMeanValue=False)
#electron_eb_2016.drawHistPlotComparison(electron_eb_2017, electron_eb_2018, nominator_name="MC", denominator_name="MC", 
#                                                                                variables=["Pt_0","Pt_1","Pt_2","Pt_3","Pt_4"], names=["2016", "2017", "2018"], markers=["<", ">"], colors=["blue", "red"],  
#                                                                                divde_by_bin_width = True, setLogy=True, setLogx=True, showMeanValue=False, ratio_max=1.45, ratio_min=0.65)

electron_ee_2016.drawHistPlotComparison(electron_ee_2017, electron_ee_2018, nominator_name="MC", denominator_name="MC", 
                                                                                variables=["Pt_0","Pt_1","Pt_2","Pt_3","Pt_4"], names=["2016", "2017", "2018"], markers=["<", ">"], colors=["blue", "red"],  
                                                                                divde_by_bin_width = True, setLogy=True, setLogx=True, showMeanValue=False, ratio_max=1.45, ratio_min=0.65)


electron_eb_2016.drawISRPlots(electron_eb_2017, electron_eb_2018, electron_ee_2016, electron_ee_2017, electron_ee_2018,
                                                          names_in_objects=["Data","Data","Data", "Data", "Data", "Data"], do_linear_fit=[False, False, False, False, False, False], 
                                                          labels=["2016 BEBE", "2017 BEBE", "2018 BEBE", "2016 EEEE", "2017 EEEE", "2018 EEEE"], 
                                                          markers=["o-","<-",">-","o","<",">"], colors=["black","blue","red","black","blue","red"], 
                                                          facecolor_list=['black','blue','red','None', 'None', 'None'], ymin=16, ymax=33)

In [ ]:
electron_2017 = isr_analysis["2017"]["electron"]["detector"]
electron_2018 = isr_analysis["2018"]["electron"]["detector"]
isr_analysis["2016"]["electron"]["detector"].drawHistPlotComparison(electron_2017, electron_2018, nominator_name="Data_Measured", denominator_name="Data_Measured", 
                                                                                variables=["Pt_0","Pt_1","Pt_2","Pt_3","Pt_4"], names=["2016", "2017", "2018"], markers=["<", ">"], colors=["blue", "red"],  
                                                                                divde_by_bin_width = True, setLogy=True, setLogx=True, showMeanValue=False, ratio_max=1.15, ratio_min=0.85)

electron_2017 = isr_analysis["2017"]["electron"]["DetUNFOLD"]
electron_2018 = isr_analysis["2018"]["electron"]["DetUNFOLD"]
isr_analysis["2016"]["electron"]["DetUNFOLD"].drawHistPlotComparison(electron_2017, electron_2018, nominator_name="Data_Measured", denominator_name="Data_Measured", 
                                                                                variables=["Pt_0","Pt_1","Pt_2","Pt_3","Pt_4"], names=["2016", "2017", "2018"], markers=["<", ">"], colors=["blue", "red"],  
                                                                                divde_by_bin_width = True, setLogy=True, setLogx=True, showMeanValue=False, ratio_max=1.45, ratio_min=0.55)

electron_2017 = isr_analysis["2017"]["electron"]["DetUNFOLD_Acceptance"]
electron_2018 = isr_analysis["2018"]["electron"]["DetUNFOLD_Acceptance"]
isr_analysis["2016"]["electron"]["DetUNFOLD_Acceptance"].drawHistPlotComparison(electron_2017, electron_2018, nominator_name="Data_Measured", denominator_name="Data_Measured", 
                                                                                variables=["Pt_0","Pt_1","Pt_2","Pt_3","Pt_4"], names=["2016", "2017", "2018"], markers=["<", ">"], colors=["blue", "red"],  
                                                                                divde_by_bin_width = True, setLogy=True, setLogx=True, showMeanValue=False)

electron_2017 = isr_analysis["2017"]["electron"]["FSRUNFOLD_Acceptance"]
electron_2018 = isr_analysis["2018"]["electron"]["FSRUNFOLD_Acceptance"]
isr_analysis["2016"]["electron"]["FSRUNFOLD_Acceptance"].drawHistPlotComparison(electron_2017, electron_2018, nominator_name="Data_Measured", denominator_name="Data_Measured", 
                                                                                variables=["Pt_0","Pt_1","Pt_2","Pt_3","Pt_4"], names=["2016", "2017", "2018"], markers=["<", ">"], colors=["blue", "red"],  
                                                                                divde_by_bin_width = True, setLogy=True, setLogx=True, showMeanValue=False)

In [ ]:
muon_2017 = isr_analysis["2017"]["muon"]["detector"]
muon_2018 = isr_analysis["2018"]["muon"]["detector"]
isr_analysis["2016"]["muon"]["detector"].drawHistPlotComparison(muon_2017, muon_2018, nominator_name="Data_Measured", denominator_name="Data_Measured", 
                                                                                variables=["Pt_0","Pt_1","Pt_2","Pt_3","Pt_4"], names=["2016", "2017", "2018"], markers=["<", ">"], colors=["blue", "red"],  
                                                                                divde_by_bin_width = True, setLogy=True, setLogx=True, showMeanValue=False, ratio_max=1.25, ratio_min=0.75)

muon_2017 = isr_analysis["2017"]["muon"]["DetUNFOLD"]
muon_2018 = isr_analysis["2018"]["muon"]["DetUNFOLD"]
isr_analysis["2016"]["muon"]["DetUNFOLD"].drawHistPlotComparison(muon_2017, muon_2018, nominator_name="Data_Measured", denominator_name="Data_Measured", 
                                                                                variables=["Pt_0","Pt_1","Pt_2","Pt_3","Pt_4"], names=["2016", "2017", "2018"], markers=["<", ">"], colors=["blue", "red"],  
                                                                                divde_by_bin_width = True, setLogy=True, setLogx=True, showMeanValue=False, ratio_max=1.45, ratio_min=0.55)

muon_2017 = isr_analysis["2017"]["muon"]["DetUNFOLD_Acceptance"]
muon_2018 = isr_analysis["2018"]["muon"]["DetUNFOLD_Acceptance"]
isr_analysis["2016"]["muon"]["DetUNFOLD_Acceptance"].drawHistPlotComparison(muon_2017, muon_2018, nominator_name="Data_Measured", denominator_name="Data_Measured", 
                                                                                variables=["Pt_0","Pt_1","Pt_2","Pt_3","Pt_4"], names=["2016", "2017", "2018"], markers=["<", ">"], colors=["blue", "red"],  
                                                                                divde_by_bin_width = True, setLogy=True, setLogx=True, showMeanValue=False)

muon_2017 = isr_analysis["2017"]["muon"]["FSRUNFOLD_Acceptance"]
muon_2018 = isr_analysis["2018"]["muon"]["FSRUNFOLD_Acceptance"]
isr_analysis["2016"]["muon"]["FSRUNFOLD_Acceptance"].drawHistPlotComparison(muon_2017, muon_2018, nominator_name="Data_Measured", denominator_name="Data_Measured", 
                                                                                variables=["Pt_0","Pt_1","Pt_2","Pt_3","Pt_4"], names=["2016", "2017", "2018"], markers=["<", ">"], colors=["blue", "red"],  
                                                                                divde_by_bin_width = True, setLogy=True, setLogx=True, showMeanValue=False)

In [ ]:
ee_data_dict_2016=isr_analysis["2016"]["muon"]["FSRUNFOLD_Acceptance"].getDict("Data")

In [ ]:
test = ee_data_dict_2016["Mass"]["Data_Measured"]["refinedUnc"].copy()

In [ ]:
test.loc[:, "content":] = test.loc[:, "content":]/ee_data_dict_2016["Mass"]["Data_Measured"]["refinedUnc"]["content"].sum()

In [ ]:
test

In [ ]:
isr_analysis["2016"]["muon"]["detector"].drawISRPlots(isr_analysis["2017"]["muon"]["detector"], isr_analysis["2018"]["muon"]["detector"],
                                                          names_in_objects=["Data","Data","Data"], do_linear_fit=[False, False, False, False, False, False], 
                                                          labels=["2016 muon", "2017 muon", "2018 muon"], marker=["o","<",">"], color_=["black","blue","red"], ymin=16, ymax=35)

In [ ]:
isr_analysis["2016"]["electron"]["detector"].drawISRPlots(isr_analysis["2017"]["electron"]["detector"], isr_analysis["2018"]["electron"]["detector"],
                                                          names_in_objects=["Data","Data","Data"], do_linear_fit=[False, False, False, False, False, False], 
                                                          labels=["2016 electron", "2017 electron", "2018 electron"], marker=["o","<",">"], color_=["black","blue","red"], ymin=16, ymax=35)

In [ ]:
isr_analysis["2016"]["muon"]["detector"].drawISRPlots(isr_analysis["2017"]["muon"]["detector"], isr_analysis["2018"]["muon"]["detector"],
                                                      names_in_objects=["DataBkgSub","DataBkgSub","DataBkgSub"], do_linear_fit=[False, False, False], labels=["2016 muon", "2017 muon", "2018 muon"])

In [ ]:
isr_analysis["2016"]["electron"]["FSRUNFOLD_Acceptance"].drawISRPlots(isr_analysis["2017"]["electron"]["FSRUNFOLD_Acceptance"], isr_analysis["2018"]["electron"]["FSRUNFOLD_Acceptance"],
                                                      names_in_objects=["Data","Data","Data"], do_linear_fit=[False, False, False], labels=["2016 electron", "2017 electron", "2018 electron"])

In [ ]:
isr_analysis["2016"]["muon"]["FSRUNFOLD_Acceptance"].drawISRPlots(isr_analysis["2017"]["muon"]["FSRUNFOLD_Acceptance"], isr_analysis["2018"]["muon"]["FSRUNFOLD_Acceptance"],
                                                      names_in_objects=["Data","Data","Data"], do_linear_fit=[False, False, False], labels=["2016 muon", "2017 muon", "2018 muon"])


In [ ]:
isr_analysis["2016"]["electron"]["FSRUNFOLD_Acceptance"].drawISRPlots(isr_analysis["2016"]["muon"]["FSRUNFOLD_Acceptance"],
                                                      names_in_objects=["Data","Data"], do_linear_fit=[True, True], labels=["2016 electron", "2016 muon"])

In [ ]:
isr_analysis["2017"]["electron"]["FSRUNFOLD_Acceptance"].drawISRPlots(isr_analysis["2017"]["muon"]["FSRUNFOLD_Acceptance"],
                                                      names_in_objects=["Data","Data"], do_linear_fit=[True, True], labels=["2017 electron", "2017 muon"])

In [ ]:
isr_analysis["2018"]["electron"]["FSRUNFOLD_Acceptance"].drawISRPlots(isr_analysis["2018"]["muon"]["FSRUNFOLD_Acceptance"],
                                                      names_in_objects=["Data","Data"], do_linear_fit=[True, True], labels=["2018 electron", "2018 muon"])

In [ ]:
isr_analysis["2016"]["electron"]["detector"].drawISRPlots(isr_analysis["2016"]["electron"]["DetUNFOLD"], isr_analysis["2016"]["electron"]["DetUNFOLD_Acceptance"],
                                                          isr_analysis["2016"]["electron"]["FSRUNFOLD"], isr_analysis["2016"]["electron"]["FSRUNFOLD_Acceptance"],
                                                          names_in_objects=["DataBkgSub","Data","Data","Data","Data"], do_linear_fit=[False, False, False, False, False], 
                                                          labels=["Detector", "Detector unfoled", "+ Efficiency", "FSR unfolded", "+ Acceptance"],
                                                          marker='o-')

In [ ]:
isr_analysis["2016"]["muon"]["detector"].drawISRPlots(isr_analysis["2016"]["muon"]["DetUNFOLD"], isr_analysis["2016"]["muon"]["DetUNFOLD_Acceptance"],
                                                          isr_analysis["2016"]["muon"]["FSRUNFOLD"], isr_analysis["2016"]["muon"]["FSRUNFOLD_Acceptance"],
                                                          names_in_objects=["DataBkgSub","Data","Data","Data","Data"], do_linear_fit=[False, False, False, False, False], 
                                                          labels=["Detector", "Detector unfoled", "+ Efficiency", "FSR unfolded", "+ Acceptance"],
                                                          marker='o-')

In [ ]:
data_dict=isr_analysis["2018"]["muon"]["FSRUNFOLD_Acceptance"].getDict("Data")
mc_dict=isr_analysis["2018"]["muon"]["FSRUNFOLD_Acceptance"].getDict("Signal")

data_dict_2016=isr_analysis["2016"]["muon"]["FSRUNFOLD_Acceptance"].getDict("Data")
mc_dict_2016=isr_analysis["2016"]["muon"]["FSRUNFOLD_Acceptance"].getDict("Signal")

data_dict_2017=isr_analysis["2017"]["muon"]["FSRUNFOLD_Acceptance"].getDict("Data")
mc_dict_2017=isr_analysis["2017"]["muon"]["FSRUNFOLD_Acceptance"].getDict("Signal")

In [ ]:
data_dict["Pt_4"]["Data_Measured"]['refinedUnc']

In [ ]:
data_dict_2016["Mass"]["Data_Measured"]['refinedUnc_meanValue']['unfold_Down']/data_dict_2016["Mass"]["Data_Measured"]['refinedUnc_meanValue']['mean'] * 100

In [ ]:
data_dict_2016["Pt_1"]["Data_Measured"]['refinedUnc_meanValue']['IdSF_Down']/data_dict_2016["Pt_1"]["Data_Measured"]['refinedUnc_meanValue']['mean'] * 100

In [ ]:
data_dict_2016["Pt_4"]["Data_Measured"]['refinedUnc_meanValue']["fsr_Down"]/data_dict_2016["Pt_4"]["Data_Measured"]['refinedUnc_meanValue']['mean'] * 100

In [ ]:
0.18282/25.068887*100

In [ ]:
ee_data_dict_2016=isr_analysis["2016"]["electron"]["FSRUNFOLD_Acceptance"].getDict("Data")

In [ ]:
ee_data_dict_2016["Pt_0"]["Data_Measured"]['refinedUnc_meanValue']

In [ ]:
0.23867/25.535478 * 100

In [ ]:
0.047467/14.527543 * 100